### One Shot Learning

Here we are checking the performance of the model trained on the English Dataset on other Datasets and their translated versions without any finetuning

## Imports

In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

from data_cleaning import Data_Preprocessing
from arabert.preprocess import ArabertPreprocessor

## Model

In [2]:
# Core
import random

# Basics
import numpy as np
import pandas as pd
import torch

# Metrics
from sklearn.metrics import *

# Tokeniser
from transformers import XLMRobertaTokenizer

# Utility
from tqdm import tqdm

# Dataloader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Scheduler
from transformers import get_linear_schedule_with_warmup

# Optimiser
from transformers import AdamW

# Model

import torch.nn as nn
from models import weighted_Roberta


class XLM_Roberta:
    def __init__(self,args):
        # fix the random
        random.seed(args['seed_val'])
        np.random.seed(args['seed_val'])
        torch.manual_seed(args['seed_val'])
        torch.cuda.manual_seed_all(args['seed_val'])
        
        # set device
        self.device = torch.device(args['device'])

        self.weights=args['weights']
        
        # initiliase tokeniser
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base', do_lower_case = True)

        self.model_save_path = args['model_save_path']
        self.name = args['name']
        
    ##-----------------------------------------------------------##
    ##----------------- Utility Functions -----------------------##
    ##-----------------------------------------------------------##
    def encode(self,data,max_len):
        input_ids = []
        attention_masks = []
        for sent in tqdm(data):
            # use in-built tokeniser of Bert
            encoded_dict = self.tokenizer.encode_plus(
                            sent,
                            add_special_tokens =True, # for [CLS] and [SEP]
                            max_length = max_len,
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,
#                             return_tensors = 'pt', # return pytorch tensors
            )
            input_ids.append(encoded_dict['input_ids'])
            # attention masks notify where padding has been added 
            # and where is the sentence
            attention_masks.append(encoded_dict['attention_mask'])
            X_data = torch.tensor(input_ids)
            attention_masks_data = torch.tensor(attention_masks)
            
        return [X_data,attention_masks_data]
    
    ##-----------------------------------------------------------##
    ##------------------ Dataloader -----------------------------##
    ##-----------------------------------------------------------##
    def get_dataloader(self,samples, batch_size,is_train=False):
        inputs,masks,labels = samples

        # Convert the lists into tensors.
#         inputs = torch.cat(inputs, dim=0)
#         masks = torch.cat(masks, dim=0)
        labels = torch.tensor(labels)

        # convert to dataset
        data = TensorDataset(inputs,masks,labels)

        if(is_train==False):
            # use random sampler for training to shuffle
            # train data
            sampler = SequentialSampler(data)
        else:
            # order does not matter for validation as we just 
            # need the metrics
            sampler = RandomSampler(data)  

        dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size,drop_last=True)

        return dataloader
    
    ##-----------------------------------------------------------##
    ##----------------- Training Utilities ----------------------##
    ##-----------------------------------------------------------## 
    def get_optimiser(self,learning_rate,model):
        # using AdamW optimiser from transformers library
        return AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8
                )
    
    def get_scheduler(self,epochs,optimiser,train_dl):
        total_steps = len(train_dl) * epochs
        return get_linear_schedule_with_warmup(optimiser, 
                num_warmup_steps = 0, 
                num_training_steps = total_steps)
    
    def evalMetric(self, y_true, y_pred, prefix):
        # calculate all the metrics and add prefix to them
        # before saving in dictionary
        accuracy = accuracy_score(y_true, y_pred)
        mf1Score = f1_score(y_true, y_pred, average='macro')
        f1Score = f1_score(y_true, y_pred)
        area_under_c = roc_auc_score(y_true, y_pred)
        recallScore = recall_score(y_true, y_pred)
        precisionScore = precision_score(y_true, y_pred)

        nonhate_f1Score = f1_score(y_true, y_pred, pos_label=0)
        non_recallScore = recall_score(y_true, y_pred, pos_label=0)
        non_precisionScore = precision_score(y_true, y_pred, pos_label=0)
        return {prefix+"accuracy": accuracy, prefix+'mF1Score': mf1Score, 
            prefix+'f1Score': f1Score, prefix+'auc': area_under_c,
            prefix+'precision': precisionScore, 
            prefix+'recall': recallScore, 
            prefix+'non_hatef1Score': nonhate_f1Score, 
            prefix+'non_recallScore': non_recallScore, 
            prefix+'non_precisionScore': non_precisionScore}
    
    ##-----------------------------------------------------------##
    ##---------------- Different Train Loops --------------------##
    ##-----------------------------------------------------------## 
    def evaluate(self,model,loader,which):
        # to evaluate model on test and validation set

        model.eval() # put model in eval mode

        # maintain total loss to save in metrics
        total_eval_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(loader):
            # separate input, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            with torch.no_grad(): # do not construct compute graph
                outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
            # output is always a tuple, thus we have to 
            # separate it manually
            #loss = outputs[0]
            logits = outputs[0]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float).to(self.device))
            
            loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))

            # add the current loss
            # loss.item() extracts loss value as a float
            total_eval_loss += loss.item()

            # calculate true labels and convert it into numpy array
            b_y_true = b_labels.cpu().data.squeeze().numpy()
            
            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

        # calculate metrics
        metrics = self.evalMetric(y_true,y_pred,which+"_")

        # Calculate the average loss over all of the batches.
        avg_loss = total_eval_loss / len(loader)
        # add it to the metric
        metrics[which+'_avg_loss'] = avg_loss

        return metrics
    
    
    def run_train_loop(self,model,train_loader,optimiser,scheduler):

        model.train() # put model in train mode

        # maintain total loss to add to metric
        total_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(train_loader):
            # separate inputs, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            # Ref: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch#:~:text=In%20PyTorch%20%2C%20we%20need%20to,backward()%20call.
            model.zero_grad()                

            outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

            # outputs is always returned as tuple
            # Separate it manually
            logits = outputs[0]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float).to(self.device))
            
            loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))
            
            # calculate current loss
            # loss.item() extracts loss value as a float
            total_loss += loss.item()

            # Back-propagation
            loss.backward()

            # calculate true labels
            b_y_true = b_labels.cpu().data.squeeze().numpy()

            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

            # clip gradient to prevent exploding gradient
            # problems
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # gradient descent
            optimiser.step()
            
            # schedule learning rate accordingly
            scheduler.step()

        # calculate avg loss 
        avg_train_loss = total_loss / len(train_loader)

        # calculate metrics
        train_metrics = self.evalMetric(y_true,y_pred,"Train_")
        
        # print results
        print('avg_train_loss',avg_train_loss)
        print('train_f1Score',train_metrics['Train_f1Score'])
        print('train_accuracy',train_metrics['Train_accuracy'])

        # add loss to metrics
        train_metrics['Train_avg_loss'] = avg_train_loss

        return train_metrics
    
    
    ##------------------------------------------------------------##
    ##----------------- Main Train Loop --------------------------##
    ##------------------------------------------------------------##
    def train(self,model,data_loaders,optimiser,scheduler,epochs,save_model):
        # save train stats per epoch
        train_stats = []
        train_loader,val_loader,test_loader = data_loaders
        # maintain best mF1 Score to save best model
        best_mf1Score=-1.0
        for epoch_i in range(0, epochs):
            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            
            print("")
            print('Training...')
            # run trian loop
            train_metrics = self.run_train_loop(model,train_loader,
                                            optimiser,scheduler)

            print("")
            print("Running Validation...") 
            # test on validation set
            val_metrics = self.evaluate(model,val_loader,"Val")
            
            print("Validation Loss: ",val_metrics['Val_avg_loss'])
            print("Validation Accuracy: ",val_metrics['Val_accuracy'])
            
            stats = {}

            # save model where validation mF1Score is best
            if(val_metrics['Val_mF1Score']>best_mf1Score):
                best_mf1Score=val_metrics['Val_mF1Score']
                if(save_model):
                    torch.save(model.state_dict(), self.model_save_path+
                        '/best_bert_'+self.name+'.pt')
                # evaluate best model on test set
                test_metrics = self.evaluate(model,test_loader,"Test")

            stats['epoch']=epoch_i+1

            # add train and val metrics of the epoch to 
            # same dictionary
            stats.update(train_metrics)
            stats.update(val_metrics)

            train_stats.append(stats)

        return train_stats,test_metrics
    
    ##-----------------------------------------------------------##
    ##----------------------- Main Pipeline ---------------------##
    ##-----------------------------------------------------------##
    def run(self,args,df_train,df_val,df_test):
        # get X and Y data points 
        X_train = df_train['Text'].values
        Y_train = df_train['Label'].values
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values
        X_val = df_val['Text'].values
        Y_val = df_val['Label'].values
        
        # encode data
        # returns list of data and attention masks
        train_data = self.encode(X_train,args['max_len'])
        val_data = self.encode(X_val,args['max_len'])
        test_data = self.encode(X_test,args['max_len'])
        
        # add labels to data so that we can send them to
        # dataloader function together
        train_data.append(Y_train)
        val_data.append(Y_val)
        test_data.append(Y_test)
        
        # convert to dataloader
        train_dl =self.get_dataloader(train_data,args['batch_size'],True)
        val_dl =self.get_dataloader(val_data,args['batch_size'])                          
        test_dl =self.get_dataloader(test_data,args['batch_size'])
        
        # intialise model
        model = weighted_Roberta.from_pretrained(
            'xlm-roberta-base', # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
            params=args['params'],
        )
        model.to(self.device)
        
        optimiser = self.get_optimiser(args['learning_rate'],model)
        
        scheduler = self.get_scheduler(args['epochs'],optimiser,train_dl)
        
        # Run train loop and evaluate on validation data set
        # on each epoch. Store best model from all epochs 
        # (best mF1 Score on Val set) and evaluate it on
        # test set
        train_stats,train_metrics = self.train(model,[train_dl,val_dl,test_dl],
                                optimiser,scheduler,args['epochs'],args['save_model'])
        
        return train_stats,train_metrics
        
    ##-----------------------------------------------------------##
    ##-------------------- Other Utilities ----------------------##
    ##-----------------------------------------------------------##
    def run_test(self,model,df_test,args):
        # to evaluate test set on the final saved model
        # to retrieve results if necessary
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values

        test_data = self.encode(X_test,args['max_len'])

        test_data.append(Y_test)

        test_dl =self.get_dataloader(test_data,32)

        metrics = self.evaluate(model,test_dl,"Test")

        return metrics
    
    def load_model(self,path,args):
        # load saved best model
        saved_model = weighted_Roberta.from_pretrained(
            'xlm-roberta-base', # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
            params=args['params'],
        )
        
        saved_model.load_state_dict(torch.load(path))
        
        return saved_model

## Utility Functions

In [3]:
def preprocess(df,isArabic):
    
    X = df['Text']
    X_new=[]
    if(isArabic):
        prep = ArabertPreprocessor('bert-base-arabertv02')
        for text in tqdm(X):
            text = prep.preprocess(text)
            X_new.append(text)
    else:
        processer = Data_Preprocessing()
        for text in tqdm(X):
            text= processer.removeEmojis(text)
            text = processer.removeUrls(text)
            text=processer.removeSpecialChar(text)
            X_new.append(text)

    df['Text']=X_new
    return df 

In [4]:
def load_dataset(args,data_path,index):
    # read dataframes
    df_test = pd.read_csv(data_path+'test_'+str(index)+'.csv')

    # clean data
    df_test=preprocess(df_test,args['isArabic'])

    return df_test

In [5]:
def one_shot_output(model_path,data_path,obj,args):
    saved_model=obj.load_model(model_path,args)
    device = torch.device(args['device'])
    saved_model=saved_model.to(device)
    
    all_metrics={}
    
    # preprocessing
    for fold in [1,2,3,4,5]:
        df = load_dataset(args,data_path,fold)

        metrics = obj.run_test(saved_model,df,args)
        
        for key,value in metrics.items():
            if(key not in all_metrics):
                all_metrics[key]=value
            else:
                all_metrics[key]+=value
    
    for key,value in all_metrics.items():
        all_metrics[key]/=5
    
    return all_metrics

### Arabic

In [16]:
DATA_PATH = "Data_Processed/Let-Mi/"
MODEL_PATH = "Saved_Models/Let-Mi/all_but_one/best_bert_xlm_roberta_3_all.pt"


model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': True,
        'model_save_path': 'Saved_Models/Let-Mi/all_but_one/',
        'isArabic': True,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

model = XLM_Roberta(model_args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,model_args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [18]:
metrics

{'Test_accuracy': 0.6279296875,
 'Test_mF1Score': 0.6101464578805273,
 'Test_f1Score': 0.5269973945013741,
 'Test_auc': 0.6252956771101191,
 'Test_precision': 0.7059106567784128,
 'Test_recall': 0.4206533390769679,
 'Test_non_hatef1Score': 0.6932955212596809,
 'Test_non_recallScore': 0.8299380151432703,
 'Test_non_precisionScore': 0.5953525767327599,
 'Test_avg_loss': 2.762599606066942}

### Italian

In [19]:
DATA_PATH = "Data_Processed/AMI-2020/"
MODEL_PATH = "Saved_Models/AMI-2020/all_but_one/best_bert_xlm_roberta_2_all.pt"


model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': True,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

model = XLM_Roberta(model_args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,model_args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [21]:
metrics

{'Test_accuracy': 0.5427898598625067,
 'Test_mF1Score': 0.535056540137524,
 'Test_f1Score': 0.4751968596074784,
 'Test_auc': 0.5390935528334275,
 'Test_precision': 0.5336107947288198,
 'Test_recall': 0.4283610136925239,
 'Test_non_hatef1Score': 0.5949162206675698,
 'Test_non_recallScore': 0.6498260919743313,
 'Test_non_precisionScore': 0.5485890780253498,
 'Test_avg_loss': 2.458373664126454}

### Hindi 

In [22]:
DATA_PATH = "Data_Processed/Shared_Task_hin/"
MODEL_PATH = "Saved_Models/Shared_Task_hin/all_but_one/best_bert_xlm_roberta_1_all.pt"


model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 4.5],
        'save_model': True,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,4.5],
            'frac':0.8
        }
    }

model = XLM_Roberta(model_args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,model_args)

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing weighted_Roberta from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of weighted_Roberta were not initialized from the model checkpoint at xlm-roberta-base and are newly initiali

In [23]:
metrics

{'Test_accuracy': 0.8106907894736842,
 'Test_mF1Score': 0.7072033568661275,
 'Test_f1Score': 0.5331543320860481,
 'Test_auc': 0.6944916196142356,
 'Test_precision': 0.5942272860851362,
 'Test_recall': 0.4842924915714734,
 'Test_non_hatef1Score': 0.8812523816462068,
 'Test_non_recallScore': 0.9046907476569981,
 'Test_non_precisionScore': 0.8590809995031726,
 'Test_avg_loss': 1.4857775659153334}

### Bengali 

In [24]:
DATA_PATH = "Data_Processed/Shared_Task_iben/"
MODEL_PATH = "Saved_Models/Shared_Task_iben/all_but_one/best_bert_xlm_roberta_5_all.pt"


model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 6.0],
        'save_model': True,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,6.0],
            'frac':0.8
        }
    }

model = XLM_Roberta(model_args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,model_args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [25]:
metrics

{'Test_accuracy': 0.8231418918918918,
 'Test_mF1Score': 0.6108749057141973,
 'Test_f1Score': 0.32349589068059503,
 'Test_auc': 0.5935847165052681,
 'Test_precision': 0.5534011023955905,
 'Test_recall': 0.22943937299347628,
 'Test_non_hatef1Score': 0.8982539207477995,
 'Test_non_recallScore': 0.9577300600170597,
 'Test_non_precisionScore': 0.8457708116643599,
 'Test_avg_loss': 1.9669636692751098}

### Spanish

In [26]:
DATA_PATH = "Data_Processed/AMI-Spanish/"
MODEL_PATH = "Saved_Models/AMI-Spanish/all_but_one/best_bert_xlm_roberta_1_all.pt"


model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': True,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

model = XLM_Roberta(model_args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,model_args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [27]:
metrics

{'Test_accuracy': 0.6125,
 'Test_mF1Score': 0.605381253486432,
 'Test_f1Score': 0.5526684318693685,
 'Test_auc': 0.6119005304875215,
 'Test_precision': 0.6490872005047186,
 'Test_recall': 0.48141229613610437,
 'Test_non_hatef1Score': 0.6580940751034957,
 'Test_non_recallScore': 0.7423887648389385,
 'Test_non_precisionScore': 0.591081139642356,
 'Test_avg_loss': 1.244543731212616}

## English

In [28]:
DATA_PATH = "Data_Processed/Shared_Task_eng/"
MODEL_PATH = "Saved_Models/Shared_Task_eng/all_but_one/best_bert_xlm_roberta_4_all.pt"


model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': True,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,8.0],
            'frac':0.8
        }
    }

model = XLM_Roberta(model_args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,model_args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [29]:
metrics

{'Test_accuracy': 0.834375,
 'Test_mF1Score': 0.6479554382504329,
 'Test_f1Score': 0.3917966071193363,
 'Test_auc': 0.6267542292292257,
 'Test_precision': 0.5371065108473771,
 'Test_recall': 0.3093664150634787,
 'Test_non_hatef1Score': 0.9041142693815294,
 'Test_non_recallScore': 0.9441420433949729,
 'Test_non_precisionScore': 0.8673895257756673,
 'Test_avg_loss': 1.865554614262838}

## Arabic Translated

In [20]:
DATA_PATH = "Data_Processed/Let-Mi_Translated/"
MODEL_PATH = "Saved_Models/Shared_Task_eng/best_bert_xlm_roberta_4_all.pt"


model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': True,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

model = XLM_Roberta(model_args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,model_args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [21]:
metrics

{'Test_accuracy': 0.5322265625,
 'Test_mF1Score': 0.39914104790283056,
 'Test_f1Score': 0.11637593692584827,
 'Test_auc': 0.5275126310640526,
 'Test_precision': 0.8916105177729203,
 'Test_recall': 0.06236579701203314,
 'Test_non_hatef1Score': 0.681906158879813,
 'Test_non_recallScore': 0.992659465116072,
 'Test_non_precisionScore': 0.5193420391499137,
 'Test_avg_loss': 2.5144972018897533}

## Italian Translated

In [8]:
DATA_PATH = "Data_Processed/AMI-2020_Translated/"
MODEL_PATH = "Saved_Models/Shared_Task_eng/best_bert_xlm_roberta_4_all.pt"


model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': True,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

model = XLM_Roberta(model_args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,model_args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [9]:
metrics

{'Test_accuracy': 0.533890798519302,
 'Test_mF1Score': 0.5103910317290101,
 'Test_f1Score': 0.4032377253327931,
 'Test_auc': 0.5274122455562283,
 'Test_precision': 0.5298040560388801,
 'Test_recall': 0.3256028353734132,
 'Test_non_hatef1Score': 0.617544338125227,
 'Test_non_recallScore': 0.7292216557390432,
 'Test_non_precisionScore': 0.535567571087997,
 'Test_avg_loss': 2.253825451683456}

## Spanish Translated

In [10]:
DATA_PATH = "Data_Processed/AMI-Spanish_Translated/"
MODEL_PATH = "Saved_Models/Shared_Task_eng/best_bert_xlm_roberta_4_all.pt"


model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': True,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "",
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

model = XLM_Roberta(model_args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,model_args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing weighted_Roberta: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing weighted_Roberta from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificati

In [11]:
metrics

{'Test_accuracy': 0.6743750000000001,
 'Test_mF1Score': 0.6701567084088194,
 'Test_f1Score': 0.7067346932684204,
 'Test_auc': 0.674585344505144,
 'Test_precision': 0.64112234595706,
 'Test_recall': 0.7876304193398093,
 'Test_non_hatef1Score': 0.6335787235492185,
 'Test_non_recallScore': 0.5615402696704787,
 'Test_non_precisionScore': 0.7276090793114998,
 'Test_avg_loss': 1.3168907338380813}

# Few Shot Learning

Here we are checking the performance of the model trained on the English Dataset on other Datasets and their translated versions with finetuning

## XLM RoBERTa Model

### Main Class

In [5]:
# Core
import random

# Basics
import numpy as np
import pandas as pd
import torch

# Metrics
from sklearn.metrics import *

# Tokeniser
from transformers import XLMRobertaTokenizer

# Utility
from tqdm import tqdm

# Dataloader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Scheduler
from transformers import get_linear_schedule_with_warmup

# Optimiser
from transformers import AdamW

# Model

import torch.nn as nn
from models import weighted_Roberta


class XLM_Roberta_fewShot:
    def __init__(self,args):
        # fix the random
        random.seed(args['seed_val'])
        np.random.seed(args['seed_val'])
        torch.manual_seed(args['seed_val'])
        torch.cuda.manual_seed_all(args['seed_val'])
        
        # set device
        self.device = torch.device(args['device'])

        self.weights=args['weights']
        
        # initiliase tokeniser
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base', do_lower_case = True)

        self.model_save_path = args['model_save_path']
        self.name = args['name']
        
    ##-----------------------------------------------------------##
    ##----------------- Utility Functions -----------------------##
    ##-----------------------------------------------------------##
    def encode(self,data,max_len):
        input_ids = []
        attention_masks = []
        for sent in tqdm(data):
            # use in-built tokeniser of Bert
            encoded_dict = self.tokenizer.encode_plus(
                            sent,
                            add_special_tokens =True, # for [CLS] and [SEP]
                            max_length = max_len,
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,
#                             return_tensors = 'pt', # return pytorch tensors
            )
            input_ids.append(encoded_dict['input_ids'])
            # attention masks notify where padding has been added 
            # and where is the sentence
            attention_masks.append(encoded_dict['attention_mask'])
            X_data = torch.tensor(input_ids)
            attention_masks_data = torch.tensor(attention_masks)
            
        return [X_data,attention_masks_data]
    
    ##-----------------------------------------------------------##
    ##------------------ Dataloader -----------------------------##
    ##-----------------------------------------------------------##
    def get_dataloader(self,samples, batch_size,is_train=False):
        inputs,masks,labels = samples

        # Convert the lists into tensors.
#         inputs = torch.cat(inputs, dim=0)
#         masks = torch.cat(masks, dim=0)
        labels = torch.tensor(labels)

        # convert to dataset
        data = TensorDataset(inputs,masks,labels)

        if(is_train==False):
            # use random sampler for training to shuffle
            # train data
            sampler = SequentialSampler(data)
        else:
            # order does not matter for validation as we just 
            # need the metrics
            sampler = RandomSampler(data)  

        dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size,drop_last=True)

        return dataloader
    
    ##-----------------------------------------------------------##
    ##----------------- Training Utilities ----------------------##
    ##-----------------------------------------------------------## 
    def get_optimiser(self,learning_rate,model):
        # using AdamW optimiser from transformers library
        return AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8
                )
    
    def get_scheduler(self,epochs,optimiser,train_dl):
        total_steps = len(train_dl) * epochs
        return get_linear_schedule_with_warmup(optimiser, 
                num_warmup_steps = 0, 
                num_training_steps = total_steps)
    
    def evalMetric(self, y_true, y_pred, prefix):
        # calculate all the metrics and add prefix to them
        # before saving in dictionary
        accuracy = accuracy_score(y_true, y_pred)
        mf1Score = f1_score(y_true, y_pred, average='macro')
        f1Score = f1_score(y_true, y_pred)
        area_under_c = roc_auc_score(y_true, y_pred)
        recallScore = recall_score(y_true, y_pred)
        precisionScore = precision_score(y_true, y_pred)

        nonhate_f1Score = f1_score(y_true, y_pred, pos_label=0)
        non_recallScore = recall_score(y_true, y_pred, pos_label=0)
        non_precisionScore = precision_score(y_true, y_pred, pos_label=0)
        return {prefix+"accuracy": accuracy, prefix+'mF1Score': mf1Score, 
            prefix+'f1Score': f1Score, prefix+'auc': area_under_c,
            prefix+'precision': precisionScore, 
            prefix+'recall': recallScore, 
            prefix+'non_hatef1Score': nonhate_f1Score, 
            prefix+'non_recallScore': non_recallScore, 
            prefix+'non_precisionScore': non_precisionScore}
    
    ##-----------------------------------------------------------##
    ##---------------- Different Train Loops --------------------##
    ##-----------------------------------------------------------## 
    def evaluate(self,model,loader,which):
        # to evaluate model on test and validation set

        model.eval() # put model in eval mode

        # maintain total loss to save in metrics
        total_eval_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(loader):
            # separate input, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            with torch.no_grad(): # do not construct compute graph
                outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
            # output is always a tuple, thus we have to 
            # separate it manually
            #loss = outputs[0]
            logits = outputs[0]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float).to(self.device))
            
            loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))

            # add the current loss
            # loss.item() extracts loss value as a float
            total_eval_loss += loss.item()

            # calculate true labels and convert it into numpy array
            b_y_true = b_labels.cpu().data.squeeze().numpy()
            
            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

        # calculate metrics
        metrics = self.evalMetric(y_true,y_pred,which+"_")

        # Calculate the average loss over all of the batches.
        avg_loss = total_eval_loss / len(loader)
        # add it to the metric
        metrics[which+'_avg_loss'] = avg_loss

        return metrics
    
    
    def run_train_loop(self,model,train_loader,optimiser,scheduler):

        model.train() # put model in train mode

        # maintain total loss to add to metric
        total_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(train_loader):
            # separate inputs, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            # Ref: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch#:~:text=In%20PyTorch%20%2C%20we%20need%20to,backward()%20call.
            model.zero_grad()                

            outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

            # outputs is always returned as tuple
            # Separate it manually
            logits = outputs[0]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float).to(self.device))
            
            loss = loss_fct(logits.view(-1, 2), b_labels.view(-1))
            
            # calculate current loss
            # loss.item() extracts loss value as a float
            total_loss += loss.item()

            # Back-propagation
            loss.backward()

            # calculate true labels
            b_y_true = b_labels.cpu().data.squeeze().numpy()

            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

            # clip gradient to prevent exploding gradient
            # problems
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # gradient descent
            optimiser.step()
            
            # schedule learning rate accordingly
            scheduler.step()

        # calculate avg loss 
        avg_train_loss = total_loss / len(train_loader)

        # calculate metrics
        train_metrics = self.evalMetric(y_true,y_pred,"Train_")
        
        # print results
        print('avg_train_loss',avg_train_loss)
        print('train_f1Score',train_metrics['Train_f1Score'])
        print('train_accuracy',train_metrics['Train_accuracy'])

        # add loss to metrics
        train_metrics['Train_avg_loss'] = avg_train_loss

        return train_metrics
    
    
    ##------------------------------------------------------------##
    ##----------------- Main Train Loop --------------------------##
    ##------------------------------------------------------------##
    def train(self,model,data_loaders,optimiser,scheduler,epochs,save_model):
        # save train stats per epoch
        train_stats = []
        train_loader,val_loader,test_loader = data_loaders
        # maintain best mF1 Score to save best model
        best_mf1Score=-1.0
        for epoch_i in range(0, epochs):
            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            
            print("")
            print('Training...')
            # run trian loop
            train_metrics = self.run_train_loop(model,train_loader,
                                            optimiser,scheduler)

            print("")
            print("Running Validation...") 
            # test on validation set
            val_metrics = self.evaluate(model,val_loader,"Val")
            
            print("Validation Loss: ",val_metrics['Val_avg_loss'])
            print("Validation Accuracy: ",val_metrics['Val_accuracy'])
            
            stats = {}

            # save model where validation mF1Score is best
            if(val_metrics['Val_mF1Score']>best_mf1Score):
                best_mf1Score=val_metrics['Val_mF1Score']
                if(save_model):
                    torch.save(model.state_dict(), self.model_save_path+
                        '/best_bert_'+self.name+'.pt')
                # evaluate best model on test set
                test_metrics = self.evaluate(model,test_loader,"Test")

            stats['epoch']=epoch_i+1

            # add train and val metrics of the epoch to 
            # same dictionary
            stats.update(train_metrics)
            stats.update(val_metrics)

            train_stats.append(stats)

        return train_stats,test_metrics
    
    ##-----------------------------------------------------------##
    ##----------------------- Main Pipeline ---------------------##
    ##-----------------------------------------------------------##
    def run(self,args,df_train,df_val,df_test):
        # get X and Y data points 
        X_train = df_train['Text'].values
        Y_train = df_train['Label'].values
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values
        X_val = df_val['Text'].values
        Y_val = df_val['Label'].values
        
        # encode data
        # returns list of data and attention masks
        train_data = self.encode(X_train,args['max_len'])
        val_data = self.encode(X_val,args['max_len'])
        test_data = self.encode(X_test,args['max_len'])
        
        # add labels to data so that we can send them to
        # dataloader function together
        train_data.append(Y_train)
        val_data.append(Y_val)
        test_data.append(Y_test)
        
        # convert to dataloader
        train_dl =self.get_dataloader(train_data,args['batch_size'],True)
        val_dl =self.get_dataloader(val_data,args['batch_size'])                          
        test_dl =self.get_dataloader(test_data,args['batch_size'])
        
        # intialise model
#         model = weighted_Roberta.from_pretrained(
#             'xlm-roberta-base', # Use the 12-layer BERT model, with an uncased vocab.
#             num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
#             params=args['params'],
#         )
        model = self.load_model(args['model_path'],args)
        model.to(self.device)
        
        optimiser = self.get_optimiser(args['learning_rate'],model)
        
        scheduler = self.get_scheduler(args['epochs'],optimiser,train_dl)
        
        # Run train loop and evaluate on validation data set
        # on each epoch. Store best model from all epochs 
        # (best mF1 Score on Val set) and evaluate it on
        # test set
        train_stats,train_metrics = self.train(model,[train_dl,val_dl,test_dl],
                                optimiser,scheduler,args['epochs'],args['save_model'])
        
        return train_stats,train_metrics
        
    ##-----------------------------------------------------------##
    ##-------------------- Other Utilities ----------------------##
    ##-----------------------------------------------------------##
    def run_test(self,model,df_test,args):
        # to evaluate test set on the final saved model
        # to retrieve results if necessary
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values

        test_data = self.encode(X_test,args['max_len'])

        test_data.append(Y_test)

        test_dl =self.get_dataloader(test_data,32)

        metrics = self.evaluate(model,test_dl,"Test")

        return metrics
    
    def load_model(self,path,args):
        # load saved best model
        saved_model = weighted_Roberta.from_pretrained(
            'xlm-roberta-base', # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
            params=args['params'],
        )
        
        saved_model.load_state_dict(torch.load(path))
        
        return saved_model

### Utility Functions

In [6]:
def load_dataset(args,index):
    # initialise constants 
    path = args['data_path']
    # read dataframes
    df_train = pd.read_csv(path+'train_'+str(index)+'.csv')
    df_val = pd.read_csv(path+'val_'+str(index)+'.csv')
    df_test = pd.read_csv(path+'test_'+str(index)+'.csv')

    # clean data
    df_train=preprocess(df_train,args['isArabic'])
    df_val=preprocess(df_val,args['isArabic'])
    df_test=preprocess(df_test,args['isArabic'])

    return df_train, df_val, df_test

In [7]:
def preprocess(df,isArabic):
    
    X = df['Text']
    X_new=[]
    if(isArabic):
        prep = ArabertPreprocessor('bert-base-arabertv02')
        for text in tqdm(X):
            text = prep.preprocess(text)
            X_new.append(text)
    else:
        processer = Data_Preprocessing()
        for text in tqdm(X):
            text= processer.removeEmojis(text)
            text = processer.removeUrls(text)
            text=processer.removeSpecialChar(text)
            X_new.append(text)

    df['Text']=X_new
    return df 

In [8]:
def save_metrics(path,metrics,which):
    df = pd.DataFrame(metrics)
    df.to_csv(path+"_"+which+".csv")

In [9]:
def fix_random(seed_val=42):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

### Main Train Function

In [10]:
def train(args, index,all_test_metrics,model_args):
    model_name = args['model_name']
    model_args['name']=model_name+'_'+str(index)+'_all'
    print("\tInitialising Model....")
    model = XLM_Roberta_fewShot(model_args)
    print("\tLoading Dataset....")
    df_train, df_val, df_test = load_dataset(args,index)
    print("\tTraining Starts....")
    train_metrics, test_metrics = model.run(model_args, 
                    df_train, df_val, df_test)

    # Save train metrics after generating path
    res_path=args['res_base_path']+model_name+'_'+model_args['name']
    save_metrics(res_path,train_metrics,"train")
    
    all_test_metrics.append(test_metrics)

### Main Run Function

In [11]:
def run(args,model_args):
    all_test_metrics=[]
    
    for fold in [1, 2, 3, 4, 5]:
        print("Fold: ",fold)
        fix_random()
        train(args,fold,all_test_metrics,model_args)
        print("Saving Test Metrics....")
        save_metrics(args['res_base_path']+args['model_name']+'_all',
                     all_test_metrics,"test")

## Arabic

In [ ]:
run_args={
    'model_name':'few_shot_xlm',
    'data_path':'Data_Processed/Let-Mi/',
    'train_cnt':'all',
    'res_base_path': 'Results/Let-Mi/fewShot/',
    'model_save_path': 'Saved_Models/Let-Mi/',
    'isArabic': True,
}

model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_path': 'Saved_Models/Let-Mi/all_but_one/best_bert_xlm_roberta_3_all.pt',
        'isArabic': True,
        'model_save_path': '',
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }
run(run_args,model_args)

Fold:  1
	Initialising Model....


/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


	Loading Dataset....


100%|██████████| 1047/1047 [00:00<00:00, 9759.42it/s]


	Training Starts....


...

100%|██████████| 65/65 [00:02<00:00, 27.60it/s]


Validation Loss:  0.45340895171348866
Validation Accuracy:  0.8461538461538461


100%|██████████| 130/130 [00:04<00:00, 27.58it/s]



======== Epoch 3 / 10 ========

Training...


 44%|████▎     | 200/458 [00:35<00:45,  5.71it/s]

## Spanish

In [ ]:
run_args={
    'model_name':'few_shot_xlm',
    'data_path':'Data_Processed/AMI-Spanish/',
    'train_cnt':'all',
    'res_base_path': 'Results/AMI-Spanish/fewShot/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_path': 'Saved_Models/AMI-Spanish/all_but_one/best_bert_xlm_roberta_1_all.pt',
        'isArabic': False,
        'model_save_path': '',
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':False,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':False,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }
run(run_args,model_args)

## Hindi

In [ ]:
run_args={
    'model_name':'few_shot_xlm',
    'data_path':'Data_Processed/Shared_Task_hin/',
    'train_cnt':'all',
    'res_base_path': 'Results/Shared_Task_hin/fewShot/',
    'model_save_path': 'Saved_Models/Shared_Task_hin/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 4.5],
        'save_model': False,
        'model_path': 'Saved_Models/Shared_Task_hin/all_but_one/best_bert_xlm_roberta_1_all.pt',
        'isArabic': False,
        'model_save_path': '',
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':False,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':False,
            'weights':[1.0,4.5],
            'frac':0.8
        }
    }
run(run_args,model_args)

## Less data points few Shot

In [12]:
def load_dataset_part(train_cnt,args,index,seed):
    # initialise constants 
    path = args['data_path']
    # read dataframes
    df_train = pd.read_csv(path+'train_'+str(index)+'.csv')
    df_val = pd.read_csv(path+'val_'+str(index)+'.csv')
    df_test = pd.read_csv(path+'test_'+str(index)+'.csv')
    
    # split train into hate and non-hate and take train_cnt
    # samples of each
    df_train_hate = df_train[df_train['Label'] == 1].sample(train_cnt,random_state=seed)
    df_train_non_hate = df_train[df_train['Label'] == 0].sample(train_cnt,random_state=seed)
    # concatenate hate and non_hate
    df_train = pd.concat([df_train_hate, df_train_non_hate])
    # shuffle the train data
    df_train = df_train.sample(frac=1).reset_index(drop=True)

    # clean data
    df_train=preprocess(df_train,args['isArabic'])
    df_val=preprocess(df_val,args['isArabic'])
    df_test=preprocess(df_test,args['isArabic'])

    return df_train, df_val, df_test

In [13]:
def train_part(args,train_cnt,run,index,all_test_metrics,model_args,seed):
    model_name = args['model_name']
    model_args['name']=model_name+'_'+str(index)+'_'+str(train_cnt)+'_'+str(run)
    print("\tInitialising Model....")
    model = XLM_Roberta_fewShot(model_args)
    print("\tLoading Dataset....")
    df_train, df_val, df_test = load_dataset_part(train_cnt,args,index,seed)
    print("\tTraining Starts....")
    train_metrics, test_metrics = model.run(model_args, 
                    df_train, df_val, df_test)

    # Save train metrics after generating path
    res_path=args['res_base_path']+model_name+'_'+model_args['name']
    save_metrics(res_path,train_metrics,"train")
    
    all_test_metrics.append(test_metrics)

In [14]:
def run_part(run_args,model_args,train_cnt):
    all_test_metrics=[]
    seeds = [42,43,44]
    for fold in [1, 2, 3, 4, 5]:
        print("Fold: ",fold)
        for run in [1,2,3]:
            print("Run: ",run)
            fix_random()
            train_part(run_args,train_cnt,run,fold,all_test_metrics,model_args,seeds[run-1])
            print("Saving Test Metrics....")
            save_metrics(run_args['res_base_path']+run_args['model_name']+
                         '_'+str(train_cnt),all_test_metrics,"test")

## Arabic few data

In [ ]:
run_args={
    'model_name':'few_shot_xlm',
    'data_path':'Data_Processed/Let-Mi/',
    'train_cnt':256,
    'res_base_path': 'Results/Let-Mi/all_but_one/',
    'model_save_path': 'Saved_Models/Let-Mi/',
    'isArabic': True,
}

model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_path': 'Saved_Models/Let-Mi/all_but_one/best_bert_xlm_roberta_3_all.pt',
        'isArabic': True,
        'model_save_path': '',
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':True,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':True,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

for train_cnt in [256,512]:
    print("Train cnt: ",train_cnt)
    run_args['train_cnt']=train_cnt
    run_part(run_args,model_args,train_cnt)

Train cnt:  256
Fold:  1
Run:  1
	Initialising Model....


/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


	Loading Dataset....


100%|██████████| 1047/1047 [00:00<00:00, 7362.82it/s]


	Training Starts....


...

100%|██████████| 64/64 [00:14<00:00,  4.51it/s]


avg_train_loss 0.10756652033956016
train_f1Score 0.9788053949903661
train_accuracy 0.978515625

Running Validation...


100%|██████████| 65/65 [00:03<00:00, 20.69it/s]


Validation Loss:  1.4385452045879972
Validation Accuracy:  0.7769230769230769

======== Epoch 8 / 10 ========

Training...


 33%|███▎      | 21/64 [00:04<00:09,  4.52it/s]

## Italian few Data

In [ ]:
run_args={
    'model_name':'few_shot_xlm',
    'data_path':'Data_Processed/AMI-2020/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-2020/all_but_one/',
    'model_save_path': 'Saved_Models/AMI-2020/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_path': 'Saved_Models/AMI-2020/all_but_one/best_bert_xlm_roberta_2_all.pt',
        'isArabic': False,
        'model_save_path': '',
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':False,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':False,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

for train_cnt in [32,64,128,256,512]:
    print("Train cnt: ",train_cnt)
    run_args['train_cnt']=train_cnt
    run_part(run_args,model_args,train_cnt)

## Spanish Few Data

In [ ]:
run_args={
    'model_name':'few_shot_xlm',
    'data_path':'Data_Processed/AMI-Spanish/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-Spanish/all_but_one/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_path': 'Saved_Models/AMI-2020/all_but_one/best_bert_xlm_roberta_2_all.pt',
        'isArabic': False,
        'model_save_path': '',
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':False,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':False,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

for train_cnt in [128,256,512]:
    print("Train cnt: ",train_cnt)
    run_args['train_cnt']=train_cnt
    run_part(run_args,model_args,train_cnt)

## Hindi Few Data

In [ ]:
run_args={
    'model_name':'few_shot_xlm',
    'data_path':'Data_Processed/Shared_Task_hin/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_hin/all_but_one/',
    'model_save_path': 'Saved_Models/Shared_Task_hin/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_path': 'Saved_Models/Shared_Task_hin/all_but_one/best_bert_xlm_roberta_1_all.pt',
        'isArabic': False,
        'model_save_path': '',
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':False,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':False,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

for train_cnt in [32,64,128,256,512]:
    print("Train cnt: ",train_cnt)
    run_args['train_cnt']=train_cnt
    run_part(run_args,model_args,train_cnt)

## Bengali Few Data

In [ ]:
run_args={
    'model_name':'few_shot_xlm',
    'data_path':'Data_Processed/Shared_Task_iben/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_iben/all_but_one/',
    'model_save_path': 'Saved_Models/Shared_Task_iben/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_path': 'Saved_Models/Shared_Task_iben/all_but_one/best_bert_xlm_roberta_5_all.pt',
        'isArabic': False,
        'model_save_path': '',
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':False,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':False,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

for train_cnt in [64,128,256,512]:
    print("Train cnt: ",train_cnt)
    run_args['train_cnt']=train_cnt
    run_part(run_args,model_args,train_cnt)

## English

In [ ]:
run_args={
    'model_name':'few_shot_xlm',
    'data_path':'Data_Processed/Shared_Task_eng/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_eng/all_but_one/',
    'model_save_path': 'Saved_Models/Shared_Task_eng/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'name': 'xlm_roberta',
        'batch_size': 8,
        'bert_model': "xlm-roberta-base",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_path': 'Saved_Models/Shared_Task_eng/all_but_one/best_bert_xlm_roberta_4_all.pt',
        'isArabic': False,
        'model_save_path': '',
        'max_length':128,
        'is_train':True,
        'epsilon':1e-8,
        'random_seed':30,
        'to_save':False,
        'frac':0.8,
        'params':{
            'max_length':128,
            'path_files': 'xlm-roberta-base',
            'what_bert':'weighted',
            'batch_size':8,
            'is_train':True,
            'learning_rate':2e-5,
            'epsilon':1e-8,
            'random_seed':30,
            'epochs':10,
            'to_save':False,
            'weights':[1.0,1.0],
            'frac':0.8
        }
    }

for train_cnt in [32,64,128,256,512]:
    print("Train cnt: ",train_cnt)
    run_args['train_cnt']=train_cnt
    run_part(run_args,model_args,train_cnt)